In [1]:
import config
import sqlalchemy
import functions
import statsapi as mlb
import sql_alch_schema
from datetime import datetime

In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,DateTime,Date,Time,Boolean
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.schema import Table
from sqlalchemy import distinct

In [3]:
_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
## Create the engine 
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True,})

In [4]:

Game = sql_alch_schema.Game
Play = sql_alch_schema.Play
Base = sql_alch_schema.Base
Team = sql_alch_schema.Team

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()

## Create the teams table
We need teams before roster because we need to map the team id pk to team id fk in roster 

In [5]:
# games_teams = Table('games_teams',Base.metadata,
#                    Column('game_id',String(150),ForeignKey('games.id')),
#                    Column('team_id',Integer,ForeignKey('teams.id')),
#                    extend_existing=True
#                    )

# Team.games = relationship("Game",secondary=games_teams,back_populates='teams')    
# Game.teams = relationship("Team",secondary=games_teams,back_populates='games')

In [6]:
class GameTeamLink(Base):
    __tablename__ = 'game_team_link'
    __table_args__ = {'extend_existing': True}
    
    game_id = Column(String(150),ForeignKey('games.id'),primary_key=True)
    team_id = Column(Integer,ForeignKey('teams.id'),primary_key=True)
    
    # add roster at the time of game 
    player_1_id = Column(Integer)
    player_2_id = Column(Integer)
    player_3_id = Column(Integer)
    player_4_id = Column(Integer)
    player_5_id = Column(Integer)
    player_6_id = Column(Integer)
    player_7_id = Column(Integer)
    player_8_id = Column(Integer)
    player_9_id = Column(Integer)
    player_10_id = Column(Integer)
    player_11_id = Column(Integer)
    player_12_id = Column(Integer)
    player_13_id = Column(Integer)
    player_14_id = Column(Integer)
    player_15_id = Column(Integer)
    player_16_id = Column(Integer)
    player_17_id = Column(Integer)
    player_18_id = Column(Integer)
    player_19_id = Column(Integer)
    player_20_id = Column(Integer)
    player_21_id = Column(Integer)
    player_22_id = Column(Integer)
    player_23_id = Column(Integer)
    player_24_id = Column(Integer)
    player_25_id = Column(Integer)
    player_26_id = Column(Integer)
    player_27_id = Column(Integer)
    player_28_id = Column(Integer)
    player_29_id = Column(Integer)
    player_30_id = Column(Integer)
    player_31_id = Column(Integer)
    player_32_id = Column(Integer)
    player_33_id = Column(Integer)
    player_34_id = Column(Integer)
    player_35_id = Column(Integer)
    player_36_id = Column(Integer)
    player_37_id = Column(Integer)
    player_38_id = Column(Integer)
    player_39_id = Column(Integer)
    player_40_id = Column(Integer)
    
    #relationships
    game = relationship('Game',back_populates='teams')
    team = relationship('Team',back_populates='games')

# update game and team tables 
Game.teams = relationship("GameTeamLink",back_populates='game')
Team.games = relationship("GameTeamLink",back_populates='team')

In [7]:
#db.execute('drop table game_team_link')
Base.metadata.create_all(db)

## Query the db to get info needed for roster table

to get the active roster from the mlb stats api, I need a team id, season, and date 

In [19]:
test_gameId = '2019/03/28/anamlb-oakmlb-1'

In [16]:
game_test = session.query(Game).filter_by(id='2019/03/28/anamlb-oakmlb-1')

In [21]:
game_test.first()homeTeam_id

SyntaxError: invalid syntax (<ipython-input-21-d622c67d9074>, line 1)

In [51]:
def create_addGameTeamLink(game_ids):
    records = []
    for game_id in game_ids:
        game_query = session.query(Game).filter_by(id=game_id)
        roster_input_dicts = get_roster_inputs(game_query)
        home_roster,away_roster = get_roster(roster_input_dicts[0])
        rosters = [home_roster,away_roster]
#       rosters.append(roster)
        
        for roster in rosters:
            game_team_record = GameTeamLink(game_id=game_id,
                                            team_id=roster['teamId'],

                                            player_1_id = roster.get('player_1','null'),
                                            player_2_id = roster.get('player_2','null'),
                                            player_3_id = roster.get('player_3','null'),
                                            player_4_id = roster.get('player_4','null'),
                                            player_5_id = roster.get('player_5','null'),
                                            player_6_id = roster.get('player_6','null'),
                                            player_7_id = roster.get('player_7','null'),
                                            player_8_id = roster.get('player_8','null'),
                                            player_9_id = roster.get('player_9','null'),
                                            player_10_id = roster.get('player_10','null'),
                                            player_11_id = roster.get('player_11','null'),
                                            player_12_id = roster.get('player_12','null'),
                                            player_13_id = roster.get('player_13','null'),
                                            player_14_id = roster.get('player_14','null'),
                                            player_15_id = roster.get('player_15','null'),
                                            player_16_id = roster.get('player_16','null'),
                                            player_17_id = roster.get('player_17','null'),
                                            player_18_id = roster.get('player_18','null'),
                                            player_19_id = roster.get('player_19','null'),
                                            player_20_id = roster.get('player_20','null'),
                                            player_21_id = roster.get('player_21','null'),
                                            player_22_id = roster.get('player_22','null'),
                                            player_23_id = roster.get('player_23','null'),
                                            player_24_id = roster.get('player_24','null'),
                                            player_25_id = roster.get('player_25','null'),
                                            player_26_id = roster.get('player_26','null'),
                                            player_27_id = roster.get('player_27','null'),
                                            player_28_id = roster.get('player_28','null'),
                                            player_29_id = roster.get('player_29','null'),
                                            player_30_id = roster.get('player_30','null'),
                                            player_31_id = roster.get('player_31','null'),
                                            player_32_id = roster.get('player_32','null'),
                                            player_33_id = roster.get('player_33','null'),
                                            player_34_id = roster.get('player_34','null'),
                                            player_35_id = roster.get('player_35','null'),
                                            player_36_id = roster.get('player_36','null'),
                                            player_37_id = roster.get('player_37','null'),
                                            player_38_id = roster.get('player_38','null'),
                                            player_39_id = roster.get('player_39','null'),
                                            player_40_id = roster.get('player_40','null')
                                           )
            records.append(game_team_record)
    return records

In [52]:
session.add_all(create_addGameTeamLink([test_gameId]))
session.commit()

In [37]:
def get_roster_inputs(query):   
    roster_inputs = []
    for instance in query.all():
        roster_input_dict = {'date':datetime.strftime(instance.dateTime,'%Y-%m-%d'),
                             'season':instance.season,
                             'homeTeam':instance.homeTeam_id,
                             'awayTeam':instance.awayTeam_id,
                             }
        roster_inputs.append(roster_input_dict)
        
    return roster_inputs
    

In [28]:
get_roster_inputs(game_test)

[{'date': '2019-03-28', 'season': 2019, 'homeTeam': 133, 'awayTeam': 108}]

In [50]:
def get_roster(roster_input_dict):
    #player_list = ['player_'+str(x) for x in range(1,41)]
    home = mlb.get('team_roster',
                   {'teamId':roster_input_dict['homeTeam'],
                    'rosterType':'active',
                    'season':roster_input_dict['season'],
                    'date':roster_input_dict['date']
                   })['roster']
    home_roster_ids = [x['person']['id'] for x in home]
    player_list = ['player_'+str(x) for x in range(1,len(home_roster_ids)+1)]
    home_roster_dict = {x:y for x,y in zip(player_list,home_roster_ids)}
    home_roster_dict['teamId'] = roster_input_dict['homeTeam']
    
    away = mlb.get('team_roster',
                   {'teamId':roster_input_dict['awayTeam'],
                    'rosterType':'active',
                    'season':roster_input_dict['season'],
                    'date':roster_input_dict['date']
                   })['roster']
    away_roster_ids = [x['person']['id'] for x in away]
    player_list = ['player_'+str(x) for x in range(1,len(away_roster_ids)+1)]
    away_roster_dict = {x:y for x,y in zip(player_list,away_roster_ids)}
    away_roster_dict['teamId']=roster_input_dict['awayTeam']
    return home_roster_dict,away_roster_dict


In [67]:
[get_roster(x) for x in roster_inputs[0:5]]

[{'player_1': 543935,
  'player_2': 592499,
  'player_3': 543278,
  'player_4': 571656,
  'player_5': 518445,
  'player_6': 596057,
  'player_7': 621385,
  'player_8': 608475,
  'player_9': 553988,
  'player_10': 572403,
  'player_11': 621592,
  'player_12': 434538,
  'player_13': 642575,
  'player_14': 606988,
  'player_15': 642397,
  'player_16': 592444,
  'player_17': 543510,
  'player_18': 600869,
  'player_19': 641729,
  'player_20': 543308,
  'player_21': 592127,
  'player_22': 519455,
  'player_23': 578428,
  'player_24': 547982,
  'player_25': 571510,
  'player_26': 605242,
  'player_27': 408234,
  'player_28': 571666,
  'player_29': 595222,
  'player_30': 543484,
  'player_31': 592206,
  'player_32': 543001,
  'player_33': 622783,
  'player_34': 642727,
  'player_35': 572888,
  'player_36': 605513,
  'player_37': 400121,
  'player_38': 622682,
  'player_39': 599683,
  'player_40': 643617},
 {'player_1': 588751,
  'player_2': 517008,
  'player_3': 552640,
  'player_4': 607455,


In [53]:
test_game = session.query(Game).filter_by(id=test_gameId)

In [58]:
test_game.first().teams

In [84]:
get_roster_inputs(test_game)

[{'date': '2019-03-31', 'season': 2019, 'homeTeam': 139, 'awayTeam': 117}]

In [85]:
roster_verify = mlb.get('team_roster',{'teamId':139,'season':2019,'date':'2019-03-31','rosterType':'active'})

In [86]:
len([x['person']['id'] for x in roster_verify['roster']])

25

In [88]:
test_roster_dict = get_roster(get_roster_inputs(test_game)[0])

In [91]:
list(test_roster_dict.values())==[x['person']['id'] for x in roster_verify['roster']]

True

In [ ]:
class Roster(Base):
    
    __tablename__ = 'rosters'
    __table_args__ = {'extend_existing': True}
    
    id = Column(String(150))
    game_id = Column(String(150),ForeignKey('games.id'))
    team.id = Column(Integer,ForeignKey('teams.id'))
    